# Installing Modules

In [ ]:
!pip install pydub
!pip install faster-whisper
!pip install streamlit

# Streamlit app.py

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from pydub import AudioSegment
import subprocess
from faster_whisper import WhisperModel
from google.colab import drive


# Function to transcribe audio and generate muted video
def generate_muted_video(file_name, excel_file):
    file_format = file_name.split('.')[-1].lower()
    if file_format == 'mp4':
        audio = AudioSegment.from_file(file_name, format="mp4")
        audio.export('/content/input_audio.wav', format="wav")
    elif file_format == 'mp3':
        audio = AudioSegment.from_file(file_name, format="mp3")
        audio.export('/content/input_audio.wav', format="wav")
    elif file_format == 'wav':
        # Use the uploaded WAV file directly
        files.move(file_name, '/content/input_audio.wav')
    else:
        raise ValueError(f"Unsupported file format: {file_format}")


    # Run WhisperModel
    model_size = "large-v2"
    model = WhisperModel(model_size, device="cuda", compute_type="float16")
    segments, _ = model.transcribe("/content/input_audio.wav", word_timestamps=True, beam_size=5)



    # word_timestamps
    t = []
    for segment in segments:
        for word in segment.words:
            t.append(word)
    csv_df = pd.DataFrame(t)

    # Remove whitespaces and punctuation
    csv_df['word'] = csv_df['word'].str.strip()
    csv_df['word'] = csv_df['word'].str.replace(r'[^\w\s]', '')

    excel_df = pd.read_excel(excel_file)
    excel_df['Keyword'] = excel_df['Keyword'].str.strip()

    # Merge the datasets on the 'word' column
    merged_df = pd.merge(csv_df, excel_df, left_on='word', right_on='Keyword', how='inner')

    merged_df.to_csv('/content/output_file.csv', index=False)

    df = pd.read_csv('/content/output_file.csv')

    ffmpeg_cmd = f'ffmpeg -i {file_name} -af "'

    for index, row in df.iterrows():
        start_time = row['start']
        end_time = row['end']
        ffmpeg_cmd += f"volume=enable='between(t,{start_time},{end_time})':volume=0, "

    # Remove the last comma and space, and close the quote
    ffmpeg_cmd = ffmpeg_cmd.rstrip(', ') + '" -c:v copy /content/output_video.mp4'

    result = subprocess.run(ffmpeg_cmd, shell=True, capture_output=True)

    # Display the muted video
    st.video('/content/output_video.mp4')


def main():
    st.title("Sound Censor")

    # Upload the video file
    uploaded_video = st.file_uploader("Upload video file", type=["mp4"])

    # Fixed path to the Excel file on Google Drive
    excel_file_path = '/content/drive/MyDrive/Dataset/Book1.xlsx'

    # Display video to the right of the sidebar
    if uploaded_video:
        st.video(uploaded_video)

    # Button to run the video muting process
    if st.button("Generate Muted Video"):
        # Save the uploaded video file
        with open('/content/input_video.' + uploaded_video.name.split('.')[-1], 'wb') as f:
            f.write(uploaded_video.read())

        # Get the uploaded file name
        video_file = '/content/input_video.' + uploaded_video.name.split('.')[-1]

        # Your logic to mute the video based on the slang dataset
        generate_muted_video(video_file, excel_file_path)


if __name__ == "__main__":
    main()

Writing app.py


# Running the website

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!npm install localtunnel
!streamlit run /content/app.py &>/content/logs.txt &
!npx localtunnel --port 8501
#34.90.24.219

Password/Enpoint IP for localtunnel is: 34.90.24.219
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 3.033s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details
npx: installed 22 in 3s
your url is: https://early-carrots-dance.loca.lt
^C
